In [1]:
import numpy as np
from scipy import linalg
import pandas as pd

In [2]:
# Read it in
path = "../data/surveys.xlsx"
survey_32W = pd.read_excel('Survey_32N and 32W consolidated.xlsx', sheet_name="Survey_32W", converters={'T3':str})
survey_32N = pd.read_excel('Survey_32N and 32W consolidated.xlsx', sheet_name="Survey_32N")

# Drop unnecessary columns
survey_32W.drop(columns=['Unnamed: 0', 'subject_id', 'image_name', 'image_name_2'], inplace=True)
survey_32N.drop(columns=['Unnamed: 0', 'subject_id', 'image_name', 'image_name_2'], inplace=True)

# Drop rows missing response
survey_32W = survey_32W[
    (survey_32W['T3'] != ' ') &
    (survey_32W['T3'] != 'none') &
    (survey_32W['T3'].notnull()) &
    (survey_32W['T3'] != 'No Comments ') &
    (survey_32W['T3'] != 0) &
    (survey_32W['T3'] != 'None') &
    (survey_32W['T3'] != '[NO ANSWER]')]

# Select responses
res_together = survey_32W['T3'][survey_32W['T1'] == "['They should be together in the same outfits']"].tolist()
res_separate = survey_32W['T3'][survey_32W['T1'] == "['They should be in separate outfits']"].tolist()

res_N = survey_32N['T5'].tolist()
long_together = survey_32W['T4'][survey_32W['T1'] == "['They should be together in the same outfits']"].tolist()
long_separate = survey_32W['T3'][survey_32W['T1'] == "['They should be in separate outfits']"].tolist()

# Importing Embedding tool

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

In [5]:
no_float = [i for i in res_N if type(i) != float]
print(len(no_float))
no_unclear = [i for i in no_float if '[unclear]' not in i]
print(len(no_unclear))

3461
2435


In [6]:
responses = pd.DataFrame({'Responses': no_unclear})

In [26]:
all_sentences = []
response_numbers = []
idx = 1
for i in responses['Responses']:
    temp = i.split('.')
    for k in temp:
        response_numbers.append(idx)
    for j in temp:
        all_sentences.append(j)
    idx += 1

In [27]:
len(all_sentences)

10510

In [28]:
len(response_numbers)

10510

In [40]:
split_responses = pd.DataFrame({'Individual_sentences': all_sentences, 'Response #': response_numbers, 'Pure_index':
                               range(0, len(all_sentences))})

In [41]:
split_responses

,Individual_sentences,Response #,Pure_index
0,Negroes have been told many times they're figh...,1,0
1,But a colored soldier is more discriminated a...,1,1
2,Its evident to those who care to see it that ...,1,2
3,Since the Emancipation Proclamation Negroes h...,1,3
4,"Each right, each privilege was fought for",1,4
...,...,...,...
10505,This his your war and I am just fighting in it,2434,10505
10506,,2434,10506
10507,I think personally this questionnaire is a goo...,2435,10507
10508,It bring out the opinion of the soldiers and ...,2435,10508


# Including DEIM implementation

In [34]:
def DEIM(V):
    #Modified Row Selection
    E = np.identity(len(V))
    Pcol = []
    p1 = np.argmax(abs(V[:,0]))
    Pcol.append(p1)
    
    for j in range(1, np.shape(V)[1]):
        Ei = np.array([E[i] for i in Pcol])
        Vi = np.transpose(np.array([V[:, i] for i in range(0, j)]))
        inv = linalg.inv(Ei @ Vi)
        P_i = Vi @ inv @ Ei
        ri = V[:, j] - P_i @ V[:, j]
        Pcol.append(np.argmax(abs(ri)))
        
    return Pcol

In [43]:
list_of_sentences = split_responses['Individual_sentences'].to_list()

In [46]:
embeddings = model.encode(list_of_sentences)

In [65]:
U, S, Vt = linalg.svd(embeddings)

AttributeError: 'list' object has no attribute 'transpose'

In [66]:
np.shape(embeddings)

(10510, 768)

In [89]:
my_matrix = np.array([list(np.array(i)) for i in embeddings])

In [96]:
U, S, Vt = linalg.svd(my_matrix)

In [ ]:
ranking = DEIM(U)